# Time Dependent Route Comparison: shadow routes

In [1]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import numpy as np
from matplotlib import pyplot as plt
from importlib import reload
import contextily as ctx
from pathlib import Path
import pandas as pd
import datetime as dt
from ipyleaflet import Map, GeoData, basemaps, LayersControl,Icon, LegendControl,ScaleControl, GeoJSON, Choropleth
from branca.colormap import linear
import json
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import ors_evaluation

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
ors_url = "https://meingruen.openrouteservice.org/ors/" 

### Routing parameters

In [6]:
profile = "foot-walking"

In [7]:
start_coords = (51.04387630433032, 13.76028997302245)
destination_coords = (51.04619669418911, 13.74123538289695)

In [8]:
body_normal = {'coordinates': [start_coords[::-1], destination_coords[::-1]],
          'instructions': 'false',
          'preference': 'recommended',
          "extra_info": ["shadow"],
          "elevation": "true",
          "continue_straight": "true",
          "options": {"avoid_features": ["ferries"], 
                      "profile_params": {"weightings": {
                          "steepness_difficulty":1, 
                          "shadow":0}}}
        
    }
body_shadow = {'coordinates': [start_coords[::-1], destination_coords[::-1]],
          'instructions': 'false',
          'preference': 'recommended',
          "extra_info": ["shadow"],
          "elevation": "true",
          "continue_straight": "true",
          "options": {"avoid_features": ["ferries"], 
                      "profile_params": {"weightings": {
                          "steepness_difficulty":1, 
                          "shadow":1}}}
    }

### Calculate initial routes

In [9]:
route_normal = ors_evaluation.Route(params=body_normal, base_url=ors_url, profile=profile, fmt="geojson")
route_shadow = ors_evaluation.Route(params=body_shadow, base_url=ors_url, profile=profile, fmt="geojson")

In [10]:
def handle_start_change(**kwargs):
    global destination_coords
    global start_coords
    global m
    start_coords = kwargs["location"]
    
    body_normal["coordinates"] = [start_coords[::-1], destination_coords[::-1]]
    body_shadow["coordinates"] = [start_coords[::-1], destination_coords[::-1]]

    route_normal = ors_evaluation.Route(params=body_normal, base_url=ors_url, profile=profile, fmt="geojson")
    route_shadow = ors_evaluation.Route(params=body_shadow, base_url=ors_url, profile=profile, fmt="geojson")
    
    m.remove_layer(m.layers[-1])
    m.remove_layer(m.layers[-1])

    route_layer_normal = GeoData(geo_dataframe = route_normal.as_dataframe(),
                            name = 'Route Normal', 
                            style={"color": "red",
                                   "opacity": 0.5})
    route_layer_shadow = GeoData(geo_dataframe = route_shadow.as_dataframe(),
                            name = 'Route Shadow',
                           style={"color": "blue",
                                   "opacity": 0.5})
    m.add_layer(route_layer_normal)
    m.add_layer(route_layer_shadow)

In [11]:
def handle_destination_change(**kwargs):
    #route = ors_evaluation.Route(params=body, base_url=ors_normal, profile=profile, fmt=fmt)
    global destination_coords
    global start_coords
    global m
    destination_coords = kwargs["location"]
    
    body_normal["coordinates"] = [start_coords[::-1], destination_coords[::-1]]
    body_shadow["coordinates"] = [start_coords[::-1], destination_coords[::-1]]

    route_normal = ors_evaluation.Route(params=body_normal, base_url=ors_url, profile=profile, fmt="geojson")
    route_shadow = ors_evaluation.Route(params=body_shadow, base_url=ors_url, profile=profile, fmt="geojson")
    
    m.remove_layer(m.layers[-1])
    m.remove_layer(m.layers[-1])

    route_layer_normal = GeoData(geo_dataframe = route_normal.as_dataframe(),
                            name = 'Route Normal', 
                            style={"color": "red",
                                   "opacity": 0.5})
    route_layer_shadow = GeoData(geo_dataframe = route_shadow.as_dataframe(),
                            name = 'Route Shadow',
                           style={"color": "blue",
                                   "opacity": 0.5})
    m.add_layer(route_layer_normal)
    m.add_layer(route_layer_shadow)

### Map showing routes

Drag the icons on the map to a different place to recalculate the routes.

In [12]:
#icon_a = Icon(icon_url='./icons/icons8-marker-a-96.png',icon_size=[25, 30], icon_anchor=[10,25])
#icon_b = Icon(icon_url='./icons/icons8-marker-b-96.png',icon_size=[25, 30], icon_anchor=[10,25])

In [13]:
from ipyleaflet import Map, Marker

center = (min(start_coords[0], destination_coords[0]) + abs(start_coords[0] - destination_coords[0]), 
          min(start_coords[1], destination_coords[1]) + abs(start_coords[1] - destination_coords[1]))

route_normal = ors_evaluation.Route(params=body_normal, base_url=ors_url, profile=profile, fmt="geojson")
route_shadow = ors_evaluation.Route(params=body_shadow, base_url=ors_url, profile=profile, fmt="geojson")

m = Map(center=center, zoom=14, scroll_wheel_zoom=True, basemap=basemaps.CartoDB.Positron)

start = Marker(location=start_coords, draggable=True)
m.add_layer(start);

destination = Marker(location=destination_coords, draggable=True)
m.add_layer(destination);

route_layer_normal = GeoData(geo_dataframe = route_normal.as_dataframe(),
                        name = 'Route Normal',
                        style={"color": "red",
                               "opacity": 0.5})
route_layer_shadow = GeoData(geo_dataframe = route_shadow.as_dataframe(),
                        name = 'Route Shadow',
                       style={"color": "blue",
                               "opacity": 0.5})

m.add_layer(route_layer_normal)
m.add_layer(route_layer_shadow)
    
start.on_move(handle_start_change)
destination.on_move(handle_destination_change)

legend = LegendControl({"Normal route": "red", "Shadow route": "blue"}, name="Legend", position="topright")
m.add_control(legend)

m.add_control(ScaleControl(position='bottomleft'))

m

Map(center=[51.04619669418911, 13.76028997302245], controls=(ZoomControl(options=['position', 'zoom_in_text', …

### Map with color by shadow value

In [31]:
color_map = linear.Reds_09

In [32]:
def compute_style(feature, colormap, choro_data):
    return {
        'color': color_map(feature['properties']['shadow_color'])
    }

In [33]:
def handle_start_change_shadow(**kwargs):
    global destination_coords
    global start_coords
    global m
    start_coords = kwargs["location"]
    
    body_normal["coordinates"] = [start_coords[::-1], destination_coords[::-1]]
    body_shadow["coordinates"] = [start_coords[::-1], destination_coords[::-1]]

    route_normal = ors_evaluation.Route(params=body_normal, base_url=ors_url, profile=profile, fmt="geojson")
    route_shadow = ors_evaluation.Route(params=body_shadow, base_url=ors_url, profile=profile, fmt="geojson")
    
    m.remove_layer(m.layers[-1])
    m.remove_layer(m.layers[-1])

    route_normal_df = route_normal.as_dataframe()
    route_normal_df["idx"] = route_normal_df.index.astype("str")
    route_normal_df["shadow_color"] = rroute_normal_df.shadow / 100.
    route_normal_json = json.loads(route_normal_df.to_json())
    shadow_values =  dict(zip(route_normal_df['idx'].tolist(), route_normal_df['shadow'].tolist()))

    layer1 = Choropleth(
        geo_data=route_normal_json,
        choro_data=shadow_values,
        colormap=linear.YlOrRd_04,
        key_on='id'
        )
    layer1.style_callback=compute_style
    m.add_layer(layer1)

    route_shadow_df = route_shadow.as_dataframe()
    route_shadow_df["idx"] = route_shadow_df.index.astype("str")
    route_shadow_df["shadow_color"] = route_shadow_df.shadow / 100.
    route_shadow_json = json.loads(route_shadow_df.to_json())
    shadow_values =  dict(zip(route_shadow_df['idx'].tolist(), route_shadow_df['shadow'].tolist()))

    layer2 = Choropleth(
        geo_data=route_shadow_json,
        choro_data=shadow_values,
        colormap=linear.YlOrRd_04,
        key_on='id'
        )
    layer2.style_callback=compute_style
    m.add_layer(layer2)

In [34]:
def handle_destination_change_shadow(**kwargs):
    #route = ors_evaluation.Route(params=body, base_url=ors_normal, profile=profile, fmt=fmt)
    global destination_coords
    global start_coords
    global m
    destination_coords = kwargs["location"]
    
    body_normal["coordinates"] = [start_coords[::-1], destination_coords[::-1]]
    body_shadow["coordinates"] = [start_coords[::-1], destination_coords[::-1]]

    route_normal = ors_evaluation.Route(params=body_normal, base_url=ors_url, profile=profile, fmt="geojson")
    route_shadow = ors_evaluation.Route(params=body_shadow, base_url=ors_url, profile=profile, fmt="geojson")
    
    m.remove_layer(m.layers[-1])
    m.remove_layer(m.layers[-1])

    route_normal_df = route_normal.as_dataframe()
    route_normal_df["idx"] = route_normal_df.index.astype("str")
    route_normal_df["shadow_color"] = route_normal_df.shadow / 100.
    route_normal_json = json.loads(route_normal_df.to_json())
    shadow_values =  dict(zip(route_normal_df['idx'].tolist(), route_normal_df['shadow'].tolist()))

    layer1 = Choropleth(
        geo_data=route_normal_json,
        choro_data=shadow_values,
        colormap=linear.YlOrRd_04,
        key_on='id'
        )
    layer1.style_callback=compute_style
    m.add_layer(layer1)

    route_shadow_df = route_shadow.as_dataframe()
    route_shadow_df["idx"] = route_shadow_df.index.astype("str")
    route_shadow_df["shadow_color"] = route_shadow_df.shadow / 100.
    route_shadow_json = json.loads(route_shadow_df.to_json())
    shadow_values =  dict(zip(route_shadow_df['idx'].tolist(), route_shadow_df['shadow'].tolist()))

    layer2 = Choropleth(
        geo_data=route_shadow_json,
        choro_data=shadow_values,
        colormap=linear.YlOrRd_04,
        key_on='id'
        )
    layer2.style_callback=compute_style
    m.add_layer(layer2)

In [35]:
from ipyleaflet import Map, Marker

center = (min(start_coords[0], destination_coords[0]) + abs(start_coords[0] - destination_coords[0]), 
          min(start_coords[1], destination_coords[1]) + abs(start_coords[1] - destination_coords[1]))

route_normal = ors_evaluation.Route(params=body_normal, base_url=ors_url, profile=profile, fmt="geojson")
route_shadow = ors_evaluation.Route(params=body_shadow, base_url=ors_url, profile=profile, fmt="geojson")

m = Map(center=center, zoom=14, scroll_wheel_zoom=True, basemap=basemaps.CartoDB.Positron)

start = Marker(location=start_coords, draggable=True)
m.add_layer(start);

destination = Marker(location=destination_coords, draggable=True)
m.add_layer(destination);

route_normal_df = route_normal.as_dataframe()
route_normal_df["idx"] = route_normal_df.index.astype("str")
route_normal_df["shadow_color"] = route_normal_df.shadow / 100.
route_normal_json = json.loads(route_normal_df.to_json())
shadow_values =  dict(zip(route_normal_df['idx'].tolist(), route_normal_df['shadow'].tolist()))

layer1 = Choropleth(
    geo_data=route_normal_json,
    choro_data=shadow_values,
    colormap=linear.YlOrRd_04,
    key_on='id'
    )
layer1.style_callback=compute_style
m.add_layer(layer1)

route_shadow_df = route_shadow.as_dataframe()
route_shadow_df["idx"] = route_shadow_df.index.astype("str")
route_shadow_df["shadow_color"] = route_shadow_df.shadow / 100.
route_shadow_json = json.loads(route_shadow_df.to_json())
shadow_values =  dict(zip(route_shadow_df['idx'].tolist(), route_shadow_df['shadow'].tolist()))

layer2 = Choropleth(
    geo_data=route_shadow_json,
    choro_data=shadow_values,
    colormap=linear.YlOrRd_04,
    key_on='id'
    )
layer2.style_callback=compute_style
m.add_layer(layer2)
    
start.on_move(handle_start_change_shadow)
destination.on_move(handle_destination_change_shadow)

legend = LegendControl({"High": "red", "Low": "#f6dfdb"}, name="Solar radiation", position="topright")
m.add_control(legend)

m.add_control(ScaleControl(position='bottomleft'))

m

Map(center=[51.04619669418911, 13.76028997302245], controls=(ZoomControl(options=['position', 'zoom_in_text', …